In [1]:
import os
import pandas as pd
import requests
import sys
sys.path.append('src')
import apilast as als
import sqltools as sqt

from PIL import Image



In [2]:
l_user = 'sinatxester'
limit = 1000
uts_num = sqt.maxuts() + 1
keylast = os.getenv("keylast")

In [3]:
 #lastf key para la api
url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&limit={limit}&from={uts_num}&api_key={keylast}&format=json'

In [4]:
req = requests.get(url).json()

In [ ]:
#requests.get(url).json()

In [ ]:
req['recenttracks']['track'][0].keys()

In [ ]:
req['recenttracks']['track']

In [ ]:
if '@attr' in req['recenttracks']['track'][0].keys():
    print(req['recenttracks']['track'][0].keys())
else:
    print(req['recenttracks']['track'][0].keys())
#exists(req['recenttracks']['track'][0]['@attr'])

In [ ]:
req['recenttracks']['track'][0].keys()

In [ ]:
requests.get(url).json()['recenttracks']['track'][0]

In [ ]:
img_src = requests.get(url).json()['recenttracks']['track'][1]['image'][3]['#text']

In [ ]:
import requests as req
from PIL import Image
from io import BytesIO


In [ ]:
response = req.get(img_src)
image = Image.open(BytesIO(response.content))
image.show()

![](imagen)


In [ ]:
#print(f'Now playing {requests.get(url).json()['recenttracks']['track'][0]['artist']} - {requests.get(url).json()['recenttracks']['track'][0]['name']}')

In [ ]:
req['recenttracks']['track'][1:] #1643911925

```PYTHON
req['recenttracks']['track'][0]
'@attr': {'nowplaying': 'true'},
```

In [ ]:
for r in req['recenttracks']['track']:
    print(r.get('date',0))


In [5]:
def req_lastfm (l_user,limit,lastuts):
    '''
    recibe parámetros para el endopint de la api de lastfm de rectents tracks de un usuario
    l_user: cadena de texto, nombre del usuario
    limit: integer del 1 a 1000 (preferible 1000) #si sabemos cuántos scrobbles hay y son menos de 1000 podemos poner un número
    lastuts: int el último uts registrado en la base de datos de mysql (fecha en segundos)
    devuelve un diccionario (json) con los últimos tracks
    '''
    uts_num = lastuts + 1 #le sumamos uno al uts recibido para no tenerlo en cuenta porque ya está en la base de datos
    keylast = os.getenv("keylast") #lastf key para la api
    url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&limit={limit}&from={uts_num}&api_key={keylast}&format=json'
    req = requests.get(url).json()
    if req['recenttracks']['@attr']['totalPages'] == '0':
        return 'No hay nuevos scrobbles'
    
    elif req['recenttracks']['@attr']['totalPages'] == '1':
        if req['recenttracks']['track'][0]['@attr'] == {'nowplaying': 'true'}:
            art_play = req['recenttracks']['track'][0]['artist']['#text']
            tit_play = req['recenttracks']['track'][0]['name']
            print(f'Now playing {art_play} by {tit_play}')
            return als.df_fromtracks(req['recenttracks']['track'][1:])
        else:
            tracks = req['recenttracks']['track']            
            return als.df_fromtracks(tracks)
    else:
        lista_lista = []
        pages = int(req['recenttracks']['@attr']['totalPages'])
        if req['recenttracks']['track'][0]['@attr'] == {'nowplaying': 'true'}:
            art_play = req['recenttracks']['track'][0]['artist']['#text']
            tit_play = req['recenttracks']['track'][0]['name']
            print(f'Now playing {art_play} by {tit_play}')
            for i in range(1,pages+1):
                url_p = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&limit={limit}&from={uts_num}&page={page}&api_key={keylast}&format=json'
                rq_p = requests.get(url_p).json()['recenttracks']['track']
                lista_lista.append(rq_p)
            tracks = [l for lista in lista_lista for l in lista]
            tracks.pop(0)
            return als.df_fromtracks(tracks)
        else:
        
            for i in range(1,pages+1):
                page = i
                url_p = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&limit={limit}&from={uts_num}&page={page}&api_key={keylast}&format=json'
                rq_p = requests.get(url_p).json()['recenttracks']['track']
                lista_lista.append(rq_p)
            tracks = [l for lista in lista_lista for l in lista]
        
            return als.df_fromtracks(tracks) #llamamos a la función df_fromtracks

In [6]:
l_user = 'sinatxester'
limit = 1000
lastuts = sqt.maxuts()
keylast = os.getenv("keylast")

In [7]:
recent = req_lastfm (l_user,limit,lastuts)

Now playing Anouk by Nobody's Wife


In [8]:
recent

,uts,artist,artist_mbid,album,album_mbid,title,track_mbid
0,1643932309,Anouk,5e8da504-c75b-4bf5-9dfc-119057c1a9c0,Together Alone,39751948-bfc3-48af-9064-094c37e94069,Nobody's Wife,0ba5ac26-3704-3142-88c1-6295a1b254c8
1,1643931996,Gloria Trevi,f5b33d1b-e8bc-444e-bc5c-490473aa0f50,Más Turbada Que Nunca,,La Papa Sin Catsup,2af9353d-fb3c-3043-adfc-4b7c5a6df92b
2,1643931802,Zaz,e3214827-bd09-4d53-a88c-893d61556352,Zaz,1780dbb3-820a-3970-a30c-b3f761543de5,Port coton,009bf74d-fb9f-3927-93d3-ab610fcb69d8
3,1643931529,Zaz,e3214827-bd09-4d53-a88c-893d61556352,Zaz,1780dbb3-820a-3970-a30c-b3f761543de5,Je veux,00f75be6-8a4f-46d2-8149-6efdee47dd75
4,1643931037,Kyla La Grange,082e714f-be12-4890-ae83-6203bd9c7d2a,Cut Your Teeth,4182ff81-da03-4941-a387-35f0479ba378,White Doves,1865d443-9f4c-4823-b857-0e336958be2c
5,1643930738,Rozalén,0432dc38-a03b-4325-9609-1caada06f47a,Con derecho a...,a72ebae6-ba4e-42cc-9c07-6efbf11f5146,80 veces,8e9ee432-6e10-3620-ad69-ee288e1fb5ef
6,1643930405,Christina Aguilera,b202beb7-99bd-47e7-8b72-195c8d72ebdd,Stripped,0c705c13-d755-4399-9ad4-a44a853ea86b,Can't Hold Us Down,1e2b9863-a38c-4a8e-819f-15fd42780feb
7,1643929919,Cutting Crew,2aa79fa9-7ac0-443b-b250-18d67227a1cb,Broadcast,07e8751b-d8be-4d09-81e6-62cd27061c7e,(I Just) Died in Your Arms,00734dfa-a187-3b54-ad65-0d8e8694edfe


In [9]:
remplazos = {"'":"\'",'\\':'\\\\','"':'\"'}
remplazos
#artist.replace(remplazos)

{"'": "'", '\\': '\\\\', '"': '"'}

In [10]:
recent.title.replace(remplazos)

0                 Nobody's Wife
1            La Papa Sin Catsup
2                    Port coton
3                       Je veux
4                   White Doves
5                      80 veces
6            Can't Hold Us Down
7    (I Just) Died in Your Arms
Name: title, dtype: object

In [ ]:
for a in artist:
    a.replace(remplazos)

In [ ]:
prueba = {'e':'ee','g':'gg'}
artist.replace(prueba)

In [ ]:
artist.replace("'","\'")

In [ ]:
def insert_data(uts, artist, artist_mbid, album, album_mbid, title, track_mbid):
    '''
    recibe los datos del datframe creado con la api de ultimos tracks de lastfm
    inserta los datos en la tabla scrobbling de mysql
    '''
    
    remplazos = {"'":"\'",'\\':'\\\\','"':'\"'}
    artist = artist.replace(remplazos)
    
    
    if '"' not in str(artist) and '"' not in str(album) and '"' not in str(title):
        try:
            engine.execute(f"""
                    INSERT INTO scrobbling (uts, artist, artist_mbid, album, album_mbid, title, track_mbid)
                    VALUES ({uts}, "{artist}", "{artist_mbid}", "{album}", "{album_mbid}", "{title}","{track_mbid}");
                    """)
        except:
            print(f'{uts} sin comillas en ningún lugar')    
            next
    elif '"' in str(artist) and '"' not in str(album) and '"' not in str(title):
        try:
            engine.execute(f"""
                    INSERT INTO scrobbling (uts, artist, artist_mbid, album, album_mbid, title, track_mbid)
                    VALUES ({uts}, '{artist}', "{artist_mbid}", "{album}", "{album_mbid}", "{title}","{track_mbid}");
                    """)
        except:
            print(f'{uts} con comillas en artista')    
            next
    elif '"' not in str(artist) and '"' in str(album) and '"' not in str(title):
        try:
            engine.execute(f"""
                    INSERT INTO scrobbling (uts, artist, artist_mbid, album, album_mbid, title, track_mbid)
                    VALUES ({uts}, "{artist}", "{artist_mbid}", '{album}', "{album_mbid}", "{title}","{track_mbid}");
                    """)
        except:
            print(f'{uts} con comillas en album')    
            next
    elif '"' not in str(artist) and '"' not in str(album) and '"' in str(title):
        try:
            engine.execute(f"""
                    INSERT INTO scrobbling (uts, artist, artist_mbid, album, album_mbid, title, track_mbid)
                    VALUES ({uts}, "{artist}", "{artist_mbid}", "{album}", "{album_mbid}", '{title}',"{track_mbid}");
                    """)
        except:
            print(f'{uts} con comillas en title')    
            next